# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Clustering via $k$-means

Last week, we studied the classification problem using the logistic regression algorithm. Since we had labels for each data point, we may regard the problem as one of _supervised learning_. However, in many applications, the data have no labels but we wish to discover possible labels (or other hidden patterns or structures). This problem is one of _unsupervised learning_. How can we approach such problems?

**Clustering** is one class of unsupervised learning methods. In this lab, we'll consider the following form of the clustering task. Suppose you are given

- a set of observations, $X \equiv \{\hat{x}_i \,|\, 0 \leq i < n\}$, and
- a target number of _clusters_, $k$.

Your goal is to partition the points into $k$ subsets, $C_0,\dots, C_{k-1} \subset X$, which are

- disjoint, i.e., $i \neq j \implies C_i \cap C_j = \emptyset$);
- but also complete, i.e., $C_0 \cup C_1 \cup \cdots \cup C_{k-1} = X$.

Intuitively, each cluster should reflect some "sensible" grouping. Thus, we need to specify what constitutes such a grouping.

## The $k$-means clustering criterion

Here is one way to measure the quality of a set of clusters. For each cluster $C$, consider its center $\mu$ and measure the distance $\|x-\mu\|$ of each observation $x \in C$ to the center. Add these up for all points in the cluster; call this sum is the _within-cluster sum-of-squares (WCSS)_. Then, set as our goal to choose clusters that minimize the total WCSS over _all_ clusters.

More formally, given a clustering $C = \{C_0, C_1, \ldots, C_{k-1}\}$, let

$$
  \mathrm{WCSS}(C) \equiv \sum_{i=0}^{k-1} \sum_{x\in C_i} \|x - \mu_i\|^2,
$$

where $\mu_i$ is the center of $C_i$. This center may be computed simply as the mean of all poitns in $C_i$, i.e.,

$$
  \mu_i \equiv \dfrac{1}{|C_i|} \sum_{x \in C_i} x.
$$

Then, our objective is to find the clustering such that

$$
  \arg\min_C \mathrm{WCSS}(C).
$$

## The standard $k$-means algorithm (Lloyd's algorithm)

Finding the global optimum is [NP-hard](https://en.wikipedia.org/wiki/NP-hardness), which is computer science mumbo jumbo for "we don't know whether there is an algorithm to calculate the exact answer in fewer steps than exponential in the size of the input." Nevertheless, there is an iterative method, Lloyd’s algorithm, that can quickly converge to a _local_ (as opposed to _global_) minimum. The procedure alternates between two operations:

**Step 1: Assignment.** Given a fixed set of $k$ centers, assign each point to the nearest center:

$$
  C_i = \{\hat{x}: \| \hat{x} - \mu_i \| \le \| \hat{x} - \mu_j \|, 1 \le j \le k \}.
$$

**Step 2: Update.** Recompute the $k$ centers ("centroids") by averaging all the data points belonging to each cluster, i.e., taking their mean:

$$
  \mu_i = \dfrac{1}{|C_i|} \sum_{\hat{x} \in C_i} \hat{x}
$$

![Illustration of $k$-means](http://stanford.edu/~cpiech/cs221/img/kmeansViz.png)

> Figure stolen without permission from: http://stanford.edu/~cpiech/cs221/img/kmeansViz.png

In the code that follows, it will be convenient to use our usual "data matrix" convention, that is, each row of a data matrix $X$ is one of $m$ observations and each column (coordinate) is one of $d$ predictors. However, we will _not_ need a dummy column of ones since we are not fitting a function per se.

$$
  X
  \equiv \left(\begin{array}{c} \hat{x}_0^T \\ \vdots \\ \hat{x}_{m}^T \end{array}\right)
  = \left(\begin{array}{ccc} x_0 & \cdots & x_{d-1} \end{array}\right).
$$

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Let's use the same dataset from the last logistic regression lab. 

In [ ]:
df = pd.read_csv ('logreg_points_train.csv')
df.head()

In [ ]:
# Helper functions from Lab 10:
def make_scatter_plot (df, x="x_1", y="x_2", hue="label",
                       palette={0: "red", 1: "olive"},
                       size=5,
                       centers=None):
    sns.lmplot (x=x, y=y, hue=hue, data=df, palette=palette,
                fit_reg=False)
    if centers is not None:
        plt.scatter (centers[:,0], centers[:,1],
                     marker=u'*', s=500,
                     c=[palette[0], palette[1]])
    
def mark_matches (a, b, exact=False):
    """
    Given two Numpy arrays of {0, 1} labels, returns a new boolean
    array indicating at which locations the input arrays have the
    same label (i.e., the corresponding entry is True).
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as the same up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    assert a.shape == b.shape
    a_int = a.astype (dtype=int)
    b_int = b.astype (dtype=int)
    all_axes = tuple (range (len (a.shape)))
    assert ((a_int == 0) | (a_int == 1)).all ()
    assert ((b_int == 0) | (b_int == 1)).all ()
    
    exact_matches = (a_int == b_int)
    if exact:
        return exact_matches

    assert exact == False
    num_exact_matches = np.sum (exact_matches)
    if (2*num_exact_matches) >= np.prod (a.shape):
        return exact_matches
    return exact_matches == False # Invert
    
def count_matches (a, b, exact=False):
    """
    Given two sets of {0, 1} labels, returns the number of mismatches.
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as similar up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    matches = mark_matches (a, b, exact=exact)
    return np.sum (matches)

In [ ]:
make_scatter_plot (df)

In [ ]:
points = df.as_matrix (['x_1', 'x_2'])
labels = df['label'].as_matrix ()
n, d = points.shape
k = 2

Note that the labels should _not_ be used in the $k$-means algorithm. We use them here only as ground truth for later verification.

### How to start? Initializing the $k$ centers

To start the algorithm, you need an initial guess. Let's randomly choose $k$ observations from the data.

**Exercise 1** (2 points). Complete the following function, which should randomly select $k$ of the given observations.

In [ ]:
def init_centers (X, k):
    """
    Randomly samples k observations from X as centers.
    Returns these centers as a (k x d) numpy array.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
centers_initial = init_centers (points, k)
print ("Initial centers:\n", centers_initial)

assert centers_initial.shape == (k, d)
assert (sum (centers_initial[0, :] == points) == [1, 1]).all ()
assert (sum (centers_initial[1, :] == points) == [1, 1]).all ()

### Computing the distances

**Exercise 2** (3 points). Implement a function that computes a distance matrix, $S = (s_{ij})$ such that $s_{ij} = d_{ij}^2$ is the _squared_ distance from point $\hat{x}_i$ to center $\mu_j$. It should return a Numpy matrix `S[:m, :k]`.

In [ ]:
def compute_d2 (X, centers):
    m = len (X)
    k = len (centers)
    
    S = np.empty((m, k))
    
    # YOUR CODE HERE
    raise NotImplementedError()

    return S

In [ ]:
centers_initial_testing = np.load ("centers_initial_testing.npy")
compute_d2_soln = np.load ("compute_d2_soln.npy")

S = compute_d2 (points, centers_initial_testing)
assert (np.linalg.norm (S - compute_d2_soln, axis=1) <= (10.0 * np.finfo(float).eps)).all ()

**Exercise 3** (2 points). Write a function that uses the (squared) distance matrix to assign a "cluster label" to each point.

That is, consider the $m \times k$ squared distance matrix $S$. For each point $i$, if $s_{i,j}$ is the minimum squared distance for point $i$, then the index $j$ is $i$'s cluster label. In other words, your function should return a (column) vector $y$ of length $m$ such that

$$
  y_i = \underset{j \in \{0, \ldots, k-1\}}{\operatorname{argmin}} s_{ij}.
$$

In [ ]:
def assign_cluster_labels (S):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
S_test1 = [[0.3, 0.2],
           [0.1, 0.5],
           [0.4, 0.2]]
y_test1 = assign_cluster_labels (S_test1)
assert (y_test1 == np.array ([1, 0, 1])).all ()

S_test2 = np.load ("assign_cluster_labels_S.npy")
y_test2_soln = np.load ("assign_cluster_labels_soln.npy")
y_test2 = assign_cluster_labels (S_test2)
assert (y_test2 == y_test2_soln).all ()

**Exercise 4** (3 points). Given a clustering (i.e., a set of points and assignment of labels), compute the center of each cluster.

In [ ]:
def update_centers (X, y):
    # X[:m, :d] == m points, each of dimension d
    # y[:m] == set of cluster labels
    m, d = X.shape
    k = max (y) + 1
    assert m == len (y)
    assert (min (y) >= 0)
    
    centers = np.empty ((k, d))
    for j in range (k):
        # Compute the new center of cluster j,
        # i.e., centers[j, :d].
        # YOUR CODE HERE
        raise NotImplementedError()
    return centers

In [ ]:
y_test3 = np.load ("y_test3.npy")
centers_test3_soln = np.load ("centers_test3_soln.npy")
centers_test3 = update_centers (points, y_test3)

delta_test3 = np.abs (centers_test3 - centers_test3_soln)
assert (delta_test3 <= 2.0*len (centers_test3_soln)*np.finfo (float).eps).all ()

**Exercise 5** (3 points). Given the squared distances, return the within-cluster sum of squares.

> _Hint_: See [numpy.amin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.amin.html#numpy.amin).

In [ ]:
def WCSS (S):
    # For example, S = [[0.3, 0.2],
    #                   [0.1, 0.5],
    #                   [0.4, 0.2]]
    # should return 0.2 + 0.1 + 0.2 = 0.5
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
S_test1 = [[0.3, 0.2],
           [0.1, 0.5],
           [0.4, 0.2]]
wcss_test1 = WCSS (S_test1)
print (wcss_test1)
assert np.abs (wcss_test1 - 0.5) <= 3.0*np.finfo (float).eps

Lastly, here is a function to check whether the centers have "moved," given two instances of the center values. It accounts for the fact that the order of centers may have changed.

In [ ]:
def has_converged (old_centers, centers):
    return set ([tuple(x) for x in old_centers]) == set ([tuple(x) for x in centers])

**Exercise 6** (4 points). Put all of the preceding building blocks together by implementing the standard $k$-means algorithm.

In [ ]:
def kmeans (X, k,
            starting_centers=None,
            max_steps=np.inf):
    if not starting_centers:
        centers = init_centers (X, k)
    else:
        centers = starting_centers
        
    converged = False
    labels = np.zeros (len (X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_centers = centers
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        print ("iteration", i, "WCSS = ", WCSS (S))
        i += 1
    return labels

clustering = kmeans (points, k)

Let's visualize the results.

In [ ]:
df['clustering'] = clustering
centers = update_centers (points, clustering)
make_scatter_plot (df, hue='clustering', centers=centers)

n_matches = count_matches (df['label'], df['clustering'])
print (n_matches,
       "matches out of",
       len (df), "possible",
       "(~ {:.1f}%)".format (100.0 * n_matches / len (df)))

## Built-in $k$-means

The preceding exercises walked you through how to implement $k$-means, but as you might have imagined, there are existing implementations as well! The following shows you how to use Scipy's implementation, which should yield similar results. If you are asked to use $k$-means in a future lab (or exam!), you can use this one.

In [ ]:
from scipy.cluster import vq

In [ ]:
# `distortion` below is the similar to WCSS.
# It is called distortion in the Scipy documentation
# since clustering can be used in compression.
centers_vq, distortion_vq = vq.kmeans (points, k)

# vq return the clustering (assignment of group for each point)
# based on the centers obtained by the kmeans function.
# _ here means ignore the second return value
clustering_vq, _ = vq.vq (points, centers_vq)

print ("Centers:\n", centers_vq)
print ("\nCompare with your method:\n", centers, "\n")
print ("Distortion (WCSS):", distortion_vq)

df['clustering_vq'] = clustering_vq
make_scatter_plot (df, hue='clustering_vq', centers=centers_vq)

n_matches_vq = count_matches (df['label'], df['clustering_vq'])
print (n_matches_vq,
       "matches out of",
       len (df), "possible",
       "(~ {:.1f}%)".format (100.0 * n_matches_vq / len (df)))